In [1]:
import numpy as np
import pandas as pd
from setting import *
import gc

gc.enable()

second_order_ratio = lambda x: x[x==1].count()/x[x==0].count()

In [2]:
priors = pd.read_csv(data + 'order_products__prior.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8}, usecols = ['order_id', 'product_id', 'reordered'], engine='c')

orders = pd.read_csv(data + 'orders.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32}, usecols = ['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow', 'order_hour_of_day'], engine='c')

product = pd.merge(priors, orders, on='order_id').sort_values(by=['user_id', 'order_number', 'product_id']).reset_index(drop=True)
product.drop('eval_set', axis=1, inplace=True)

In [4]:
t3 = product[['user_id', 'order_hour_of_day', 'order_id']][product.reordered==1].\
groupby(by=['user_id', 'order_hour_of_day'])['order_id'].\
agg({'user_hour_count': 'count'}).reset_index()
t31 = product[['user_id', 'order_hour_of_day', 'order_id']][product.reordered==1].\
groupby(by=['user_id'])['order_id'].\
agg({'user_count': 'count'}).reset_index()
t3 = t3.merge(t31, on='user_id')
t3['hour_user_reorder_prob'] = (t3.user_hour_count+1)/(t3.user_count+24)
t3.drop(['user_hour_count', 'user_count'], axis=1, inplace=True)

In [6]:
product_detail = pd.read_csv(data+"products.csv", dtype = {
    'product_id': np.uint16,
    'product_name': str,
    'aisle_id': np.uint8,
    'department_id': np.uint8
}, usecols = ['product_id', 'aisle_id', 'department_id'])

priors = pd.read_csv(data + 'order_products__prior.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8}, usecols = ['order_id', 'product_id', 'reordered'], engine='c')

orders = pd.read_csv(data + 'orders.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32}, usecols = ['order_id', 'user_id', 'eval_set', 'order_number'], engine='c')

product = pd.merge(priors, orders, on='order_id').sort_values(by=['user_id', 'order_number', 'product_id']).reset_index(drop=True)
product.drop('eval_set', axis=1, inplace=True)

In [7]:
d = pd.merge(product_detail, product[['product_id', 'user_id', 'order_id']][product.reordered==1], on='product_id')

d2 = d[['user_id', 'order_id']].groupby(by='user_id')['order_id'].agg({'user_rorder_cnt': 'count'}).reset_index()
d21 = d[['user_id', 'aisle_id', 'order_id']].groupby(by=['user_id', 'aisle_id'])['order_id'].agg({'aisle_count': 'count'}).reset_index()
d22 = d[['user_id', 'department_id', 'order_id']].groupby(by=['user_id', 'department_id'])['order_id'].agg({'department_count': 'count'}).reset_index()
d2 = d2.merge(d21, on='user_id').merge(d22, on='user_id')
d2['aisle_user_reorder_prob'] = d2.aisle_count/d2.user_rorder_cnt
d2['department_user_reorder_prob'] = d2.department_count/d2.user_rorder_cnt
d2.drop(['user_rorder_cnt', 'aisle_count', 'department_count'], axis=1, inplace=True)

In [9]:
d21

,user_id,aisle_id,aisle_count
0,1,21,7
1,1,23,10
2,1,24,1
3,1,53,1
4,1,54,1
5,1,77,11
6,1,117,8
7,1,121,2
8,2,21,2
9,2,23,8


In [11]:
d22

,user_id,department_id,department_count
0,1,4,1
1,1,7,11
2,1,14,2
3,1,16,8
4,1,17,1
5,1,19,18
6,2,1,5
7,2,4,20
8,2,6,1
9,2,7,3
